In [4]:
!pip install deepface #emotion, age, gender, race

In [14]:
%%writefile Human_Emotion_Rec.py
import streamlit as st
import numpy as np
import cv2
from deepface import DeepFace
from PIL import Image
import tempfile
st.title('Human Emotion Recognition Application')

st.write('Upload an Image or Video')

option = st.selectbox("choose an option", ('Image', 'Video'))

# define function to analyze image or video to extract emotion
def Analyze_Emotion(image_or_video):
    try:
        analysis = DeepFace.analyze(image_or_video, actions = ['emotion'], enforce_detection = False)
        return analysis[0]['emotion']
    except ValueError as e:
        st.write(f"Error: {e}")
        return None

# to analyze image
if option == 'Image':
    uploaded_image = st.file_uploader("Please, Upload an Image To Analyze", type = ['jpg','png','jpeg'])
    if uploaded_image is not None:
        img = Image.open(uploaded_image)
        img_array = np.array(img)
        st.image(img_array, channels = 'RGB')
        #detect emotion 
        emotion_Scores = Analyze_Emotion(img_array)
        if emotion_Scores:
            detected_emotion = max(emotion_Scores, key = emotion_Scores.get)
            st.write(f'Detected Emotion: {detected_emotion}')
        else:
            st.write('NO Face Detected in Your Image')

#to analyze video
if option == 'Video':
    upload = st.file_uploader("Please, Upload Video to Analyze", type = ['mp4', 'mov', 'avi'])
    if upload is not None:
        with tempfile.NamedTemporaryFile(delete = False) as temp_video:
            temp_video.write(upload.read())
            video_path = temp_video.name
        video = cv2.VideoCapture(video_path)

        frame_rate = 20 #control how many frames
        frame_count = 0

        while video.isOpened():
            ret, frame = video.read()
            if not ret:
                break
            frame_count += 1

            #process and display every time
            if frame_count % frame_rate == 0:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                emotion_scores = Analyze_Emotion(frame_rgb)
                if emotion_scores:
                    detected_emotion = max(emotion_scores, key = emotion_scores.get)
                    cv2.putText(frame, detected_emotion, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)
                else:
                    detected_emotion = 'No Face Detected'
                    cv2.putText(frame, detected_emotion, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)

                st.image(frame, channels = 'BGR')
        video.release()

Overwriting Human_Emotion_Rec.py
